# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=10, hover_cols=['City'], tiles = 'OSM', color = 'purple')

# Display the map
map_plot.opts(global_extent=True, width=700, height=400)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

min_temp_celsius = (65 - 32) * 5.0/9.0
max_temp_celsius = (85 - 32) * 5.0/9.0

temp_filtered_df = city_data_df[
    (city_data_df['Max Temp'] <= max_temp_celsius) & 
    (city_data_df['Max Temp'] >= min_temp_celsius)
]

# Drop any rows with null values
temp_filtered_df = temp_filtered_df.dropna()
# Display sample data
temp_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
10,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
...,...,...,...,...,...,...,...,...,...,...
563,563,ilhabela,-23.7781,-45.3581,22.67,88,95,2.32,BR,1666108558
566,566,nhulunbuy,-12.2333,136.7667,26.34,78,88,0.00,AU,1666108559
567,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559
569,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = temp_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
10,colorado,BR,-22.8375,-51.9731,86,
19,puerto ayora,EC,-0.7393,-90.3518,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Search within 10,000 meters
params = {
    "apiKey": geoapify_key, # API key
    "categories": "accommodation.hotel",  # Specify the category of the place as hotel
    "limit": 1,  # Limit to one result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"

    # Make an API request using the params dictionary
    response = requests.get("https://api.geoapify.com/v2/places", params=params)
    
    # Check if the response was successful before continuing
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"Failed to get a response for the city: {row['City']}, status code: {response.status_code}")
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head())

Starting hotel search
faya - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: La Villa Delisle
colorado - nearest hotel: Servilha Park Hotel
puerto ayora - nearest hotel: Deja Vú
avarua - nearest hotel: Natura Inano Villa
rikitea - nearest hotel: Chez Bianca & Benoit
bambous virieux - nearest hotel: Preskil Island Resort
cape town - nearest hotel: Southern Sun Cape Sun
naberera - nearest hotel: No hotel found
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Mision San Miguel
sao joao da barra - nearest hotel: Unidade Hoteleira Sesc Grussaí
kapaa - nearest hotel: Sheraton Kauai Coconut Beach Resort
mogadishu - nearest hotel: Elite Hotel
hilo - nearest hotel: Dolphin Bay Hotel
businga - nearest hotel: Hôtel Roho
lasa - nearest hotel: Gasthof Neue Post
vaini - nearest hotel: Emerald Hotel
east london - nearest hotel: Garden Court East London
banda - nearest hotel: #acnindiafy21
atuona - nearest hotel: Pearl Resort

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotels_found_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]


map_with_hotels = hotels_found_df.hvplot.points(
    'Lng', 'Lat', geo=True, size=10,
    hover_cols=['City', 'Hotel Name', 'Country'], tiles = 'OSM', color = 'purple'
)

# Display the map
map_with_hotels.opts(global_extent=True, width=700, height=400)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)